In [1]:
spark.version

2.4.8

In [2]:
import java.sql.Date
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

In [5]:
val purchasesDF = Seq(
    (Date.valueOf("2019-01-01"), "01"),
    (Date.valueOf("2019-05-10"), "01"),
    (Date.valueOf("2019-03-05"), "02"),
    (Date.valueOf("2019-02-20"), "03"),
    (Date.valueOf("2019-01-20"), "02")
).toDF("purchase_date", "device_id")

purchasesDF = [purchase_date: date, device_id: string]


[purchase_date: date, device_id: string]

In [6]:
val devicesDF = Seq(
    ("01", "notebook", 600.00),
    ("02", "small phone", 100.00),
    ("03", "camera",150.00),
    ("04", "large phone", 700.00)
).toDF("device_id", "device_name", "price")

devicesDF = [device_id: string, device_name: string ... 1 more field]


[device_id: string, device_name: string ... 1 more field]

### error message example

In [63]:
%%dataframe
purchasesDF.join(devicesDF, col("device_id") === col("device_id"), "inner")

lastException: Throwable = null


An error occurred converting DataFrame to html.
Reference 'device_id' is ambiguous, could be: device_id, device_id.;

## adding aliases

In [68]:
%%dataframe
%%scan
purchasesDF.as("purchases")
  .join(devicesDF.as("devices"), col("purchases.device_id") === col("devices.device_id"))

purchase_date,device_id,device_id,device_name,price
2019-05-10,01,01,notebook,600.0
2019-01-01,01,01,notebook,600.0
2019-01-20,02,02,small phone,100.0
2019-03-05,02,02,small phone,100.0
2019-02-20,03,03,camera,150.0


## Viewing qualified column names via exceptions

An engineer's gotta do what an engineer's gotta do

In [60]:
purchasesDF.as("purchases")
  .join(devicesDF.as("devices"), col("purchases.device_id") === col("devices.device_id"))
  .select("xxxx")

lastException: Throwable = null


An error occurred converting DataFrame to html.
cannot resolve '`xxxx`' given input columns: [purchases.device_id, devices.price, devices.device_name, purchases.purchase_date, devices.device_id];;
'Project ['xxxx]
+- Join Inner, (device_id#20 = device_id#30)
   :- SubqueryAlias `purchases`
   :  +- Project [_1#16 AS purchase_date#19, _2#17 AS device_id#20]
   :     +- LocalRelation [_1#16, _2#17]
   +- SubqueryAlias `devices`
      +- Project [_1#26 AS device_id#30, _2#27 AS device_name#31, _3#28 AS price#32]
         +- LocalRelation [_1#26, _2#27, _3#28]


## Dropping

In [66]:
%%dataframe
%%scan
purchasesDF.as("purchases")
  .join(devicesDF.as("devices"), col("purchases.device_id") === col("devices.device_id"))
  .drop("device_id")

purchase_date,device_name,price
2019-05-10,notebook,600.0
2019-01-01,notebook,600.0
2019-01-20,small phone,100.0
2019-03-05,small phone,100.0
2019-02-20,camera,150.0
